In [1]:
import networkx as nx
import functools

with open('./inputs/day21.txt', 'r') as f:
    final_codes = list(map(lambda x: x.strip(), f.readlines()))

keypad_graph = nx.DiGraph()
keypad_graph.add_edges_from([
    ('7', '4', {'direction': 'v'}), ('8', '5', {'direction': 'v'}), ('9', '6', {'direction': 'v'}),
    ('4', '1', {'direction': 'v'}), ('5', '2', {'direction': 'v'}), ('6', '3', {'direction': 'v'}),
    ('2', '0', {'direction': 'v'}), ('3', 'A', {'direction': 'v'}),
    ('0', '2', {'direction': '^'}), ('A', '3', {'direction': '^'}), ('1', '4', {'direction': '^'}),
    ('2', '5', {'direction': '^'}), ('3', '6', {'direction': '^'}), ('4', '7', {'direction': '^'}),
    ('5', '8', {'direction': '^'}), ('6', '9', {'direction': '^'}),
    ('7', '8', {'direction': '>'}), ('4', '5', {'direction': '>'}), ('1', '2', {'direction': '>'}),
    ('8', '9', {'direction': '>'}), ('5', '6', {'direction': '>'}), ('2', '3', {'direction': '>'}),
    ('0', 'A', {'direction': '>'}),
    ('9', '8', {'direction': '<'}), ('6', '5', {'direction': '<'}), ('3', '2', {'direction': '<'}),
    ('A', '0', {'direction': '<'}), ('8', '7', {'direction': '<'}), ('5', '4', {'direction': '<'}),
    ('2', '1', {'direction': '<'}),
])

arrow_graph = nx.DiGraph()
arrow_graph.add_edges_from([
    ('^', 'v', {'direction': 'v'}), ('A', '>', {'direction': 'v'}),
    ('v', '^', {'direction': '^'}), ('>', 'A', {'direction': '^'}),
    ('<', 'v', {'direction': '>'}), ('^', 'A', {'direction': '>'}), ('v', '>', {'direction': '>'}),
    ('A', '^', {'direction': '<'}), ('>', 'v', {'direction': '<'}), ('v', '<', {'direction': '<'}),
])

@functools.cache
def get_arrow_num_steps(start_button, end_button, num_inception_bots):
    if num_inception_bots == 0:
        return 1
    else:
        m = float('inf')
        for p in nx.all_shortest_paths(arrow_graph, start_button, end_button):
            path_directions = [arrow_graph.get_edge_data(p[i], p[i+1])['direction'] for i in range(len(p) - 1)] + ['A']
            path_length_idk = get_arrow_num_steps(
                start_button='A',
                end_button=path_directions[0],
                num_inception_bots=num_inception_bots-1
            )
            for i in range(len(path_directions) - 1):
                path_length_idk += get_arrow_num_steps(
                    start_button=path_directions[i],
                    end_button=path_directions[i+1],
                    num_inception_bots=num_inception_bots-1
                )

            m = min(m, path_length_idk)

        return m

def get_num_num_steps(start_num, end_num, num_inception_bots):
    m = float('inf')
    for p in nx.all_shortest_paths(keypad_graph, start_num, end_num):
        path_directions = [keypad_graph.get_edge_data(p[i], p[i+1])['direction'] for i in range(len(p) - 1)] + ['A']
        path_length_idk = get_arrow_num_steps(
            start_button='A',
            end_button=path_directions[0],
            num_inception_bots=num_inception_bots
        )
        for i in range(len(path_directions) - 1):
            path_length_idk += get_arrow_num_steps(
                start_button=path_directions[i],
                end_button=path_directions[i+1],
                num_inception_bots=num_inception_bots
            )

        m = min(m, path_length_idk)

    return m

def get_code_complexity(keypad_code, num_inception_bots):
    keypad_code = 'A' + keypad_code
    final_code_length = sum(get_num_num_steps(keypad_code[i], keypad_code[i+1], num_inception_bots) for i in range(len(keypad_code) - 1))
    return final_code_length * int(keypad_code[1:-1])

print('Answer to Day 21, Part 1:', sum(get_code_complexity(c, 2) for c in final_codes))
print('Answer to Day 21, Part 2:', sum(get_code_complexity(c, 25) for c in final_codes))

Answer to Day 21, Part 1: 270084
Answer to Day 21, Part 2: 329431019997766


In [2]:
import torch

with open('./inputs/day22.txt', 'r') as f:
    initial_numbers = list(map(int, f.readlines()))

def evolve_secret(secret_num, n=1):
    for _ in range(n):
        secret_num = (secret_num ^ (secret_num * 64)) % 16777216
        secret_num = (secret_num ^ int(secret_num / 32)) % 16777216
        secret_num = (secret_num ^ secret_num * 2048) % 16777216

    return secret_num

print('Answer to Day 22, Part 1:', sum(evolve_secret(n, 2000) for n in initial_numbers))

def get_prices(secret):
    return torch.tensor([int(str(secret)[-1:])] + [int(str(secret := evolve_secret(secret))[-1:]) for _ in range(1999)], dtype=int)

all_sequences = {}
for n in initial_numbers:
    d = {}
    prices = get_prices(n)
    diffs = prices.diff()
    for i in range(len(diffs) - 3):
        t = tuple(diffs[i: i+4].tolist())
        if not t in d:
            d[t] = int(prices[i+4])

    all_sequences[n] = d

all_keys = set()
for init_number in initial_numbers:
    all_keys = all_keys.union(all_sequences[init_number].keys())

best_score = 0
for k in all_keys:
    score = sum(all_sequences[n].get(k, 0) for n in initial_numbers)
    best_score = max(score, best_score)

print('Answer to Day 22, Part 2:', best_score)

Answer to Day 22, Part 1: 18694566361
Answer to Day 22, Part 2: 2100
